# Credentials of LLM provider

1. API_ENDPOINT: Exposed URL of LLM provider
2. AUTH_KEY: Secret key that authenticates API requests.
3. DEFAULT_MODEL: Default model to use

In [ ]:
API_ENDPOINT = ""
AUTH_KEY = "sk-hVV5pRvA8T7ZYN6pO9_ZkA"
DEFAULT_MODEL = "cerebras/qwen-3-32b"

Data Schema

In [118]:
from typing import TypedDict, Literal, Optional, List

class Message(TypedDict):
    """
    Schema for a single message.

    Attributes:
        - role: Any one of system, user or ai
    """
    role: Literal["system", "user", "ai"]
    message: str


class Conversation(TypedDict):
    messages: Optional[List[Message]]

In [119]:
message = Message(
    role = "human",
    content = "Hi, how are you?"
)

print(message)

{'role': 'human', 'content': 'Hi, how are you?'}


In [120]:
conversation = Conversation(
    messages=[
        Message(role="system", content="You are a helpful assistant named ChatGPT"),
        Message(role="user", content="Hi, How are you?")
    ]
)

print(conversation)

{'messages': [{'role': 'system', 'content': 'You are a helpful assistant named ChatGPT'}, {'role': 'user', 'content': 'Hi, How are you?'}]}


Function to make LLM calls

In [121]:
import httpx

In [ ]:
def call_llm(conversation: Conversation):
    """
    Sends an API request to LLM Provider

    Parameters:
        - conversation: Conversation History

    Returns:
        - response: Raw response from Provider
    """
    # Prepare Request Body
    body = {
        "model": DEFAULT_MODEL,
        "messages": conversation["messages"]
    }

    # Headers
    headers = {
        "Authorization": f"Bearer {AUTH_KEY}",
        "Content-Type": "application/json"
    }

    with httpx.Client() as client:
        response = client.post(
            url = API_ENDPOINT,
            json = body,
            headers = headers,
            timeout = 600 # seconds
        )

    response.raise_for_status()

    return response


def preprocess(system_prompt: str, user_prompt: str):
    """
    Takes input a system prompt and user prompt and
    creates a conversation history.
    """
    conversation = Conversation(
        messages=[
            Message(role="system", content=system_prompt),
            Message(role="user", content=user_prompt)
        ]
    )
    return conversation


def postprocess(response: httpx.Response):
    """
    Extracts the LLM response from the raw response
    from the LLM Provider.
    """
    response = response.json()
    message = Message(
        role = "ai",
        content = response["choices"][0]["message"]["content"]
    )
    return message

Start a conversation

In [123]:
system_prompt = """
You are a helpful AI assistant.
"""

user_prompt = "Help me write a simple python code to print first N prime numbers using recursion"

In [124]:
# create a conversation
conversation = preprocess(system_prompt, user_prompt)

# send conversation to LLM
api_response = call_llm(conversation)

# postprocess the response
llm_response = postprocess(api_response)

In [125]:
print(llm_response)

{'role': 'ai', 'content': '\n\nHere\'s a well-documented Python code to print the first `N` prime numbers using recursion:\n\n---\n\n```python\ndef first_n_primes(n):\n    def is_prime(num, i=2):\n        """\n        Recursively checks if a number is prime by testing divisibility \n        from 2 up to the square root of the number.\n        """\n        if i * i > num:\n            return True\n        if num % i == 0:\n            return False\n        return is_prime(num, i + 1)\n\n    def helper(n, current):\n        """\n        Recursive helper function to build a list of the first N primes.\n        - Base case: when n == 0, return an empty list.\n        - If the current number is prime, add it as the first element \n          and call helper with n-1 for the next prime.\n        - If not a prime, continue checking the next number.\n        """\n        if n == 0:\n            return []  # Base case: no more primes to collect\n        if is_prime(current):\n            return 